In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from scipy.stats import binom
from typing import List, Tuple

In [ ]:
def log_like_bin(x: float, n: float, theta: List[float]) -> float:
    log_like = binom.logpmf(x, n, theta)
    log_like = log_like - np.max(log_like)
    return log_like

def bin_mle(x, n):
    mle = x / n
    return mle

def bin_obs_fisher_info(n, mle):
    obs_fisher_info = n / (mle * (1 - mle))
    return obs_fisher_info

def bin_obs_fisher_info_trans(x, n):
    obs_fisher_info = x * (n - x) / n
    return obs_fisher_info

def approx_log_like_bin(x, n, theta: List[float]) -> float:
    theta_hat = bin_mle(x, n)
    i_theta_hat = bin_obs_fisher_info(n, theta_hat)
    log_like = - 0.5 * i_theta_hat * (theta - theta_hat)**2 
    return log_like

def approx_log_like_bin_trans(x, n, theta: List[float]) -> float:
    psi = np.log(theta) - np.log(1 - theta)
    theta_hat = bin_mle(x, n)
    psi_hat = np.log(theta_hat) - np.log(1 - theta_hat)
    i_psi_hat = bin_obs_fisher_info_trans(x, n)
    log_like = - 0.5 * i_psi_hat * (psi - psi_hat)**2 
    return log_like

In [ ]:
x = 8
n = 10
theta = np.linspace(0.4, 0.99, num=100)
log_like = log_like_bin(x, n, theta)
approx_log_like = approx_log_like_bin(x, n, theta)

In [ ]:
plt.plot(theta, log_like)
plt.plot(theta, approx_log_like, '--')
plt.axhline(y=np.log(0.15))
plt.xlabel(r'$\theta$')
plt.ylabel('Log-likelihood')
plt.title('Probability scale');

In [ ]:
def likelihood_interval(theta: List[float],
                        likelihood: List[float],
                        cutoff: float) -> Tuple[float, float]:
    # intersection points occur below and above the maximum likelihood estimate
    mle_index = np.argmax(likelihood)
    interp_below_max = interp1d(likelihood[:mle_index], theta[:mle_index])
    interp_above_max = interp1d(likelihood[mle_index:], theta[mle_index :])
    lower_int = np.round(interp_below_max(cutoff).flatten()[0], 2)
    upper_int = np.round(interp_above_max(cutoff).flatten()[0], 2)
    return (lower_int, upper_int)

In [ ]:
c = 0.15
print(f'Likelihood interval for c = {c} is {likelihood_interval(theta, np.exp(log_like), c)}') 

In [ ]:
psi = np.log(theta) - np.log(1 - theta)
approx_log_like = approx_log_like_bin_trans(x, n, theta)

In [ ]:
plt.plot(psi, log_like)
plt.plot(psi, approx_log_like, '--')
plt.axhline(y=np.log(0.15))
plt.ylim((-4.5, 0.1))
plt.xlabel(r'$\psi$')
plt.ylabel('Log-likelihood')
plt.title('Log-odds scale');